## Running seisflows following 2D Example Walkthrough
https://seisflows.readthedocs.io/en/devel/2D_example_walkthrough.html   
by AR  

In [ ]:
import os
import glob
import shutil
import numpy as np
from IPython.display import Image

In [ ]:
# vvv USER MUST EDIT THE FOLLOWING PATHS vvv
WORKDIR = os.getcwd()
SPECFEM2D = "/home/scoped"
# where WORKDIR: points to your own working directory
# and SPECFEM2D: points to an existing specfem2D repository if available (if not set as '')
# ^^^ USER MUST EDIT THE FOLLOWING PATHS ^^^
# ======================================================================================================

# Distribute the necessary file structure of the SPECFEM2D repository that we will downloaded/reference
SPECFEM2D_ORIGINAL = os.path.join(SPECFEM2D, "specfem2d") 
SPECFEM2D_BIN_ORIGINAL = os.path.join(SPECFEM2D_ORIGINAL, "bin")
SPECFEM2D_DATA_ORIGINAL = os.path.join(SPECFEM2D_ORIGINAL, "DATA")
TAPE_2007_EXAMPLE = os.path.join(SPECFEM2D_ORIGINAL, "EXAMPLES", "Tape2007")

# The SPECFEM2D working directory that we will create separate from the downloaded repo
SPECFEM2D_WORKDIR = os.path.join(WORKDIR, "specfem2d_workdir")
SPECFEM2D_BIN = os.path.join(SPECFEM2D_WORKDIR, "bin")
SPECFEM2D_DATA = os.path.join(SPECFEM2D_WORKDIR, "DATA")
SPECFEM2D_OUTPUT = os.path.join(SPECFEM2D_WORKDIR, "OUTPUT_FILES")

# Pre-defined locations of velocity models we will generate using the solver
SPECFEM2D_MODEL_INIT = os.path.join(SPECFEM2D_WORKDIR, "OUTPUT_FILES_INIT")
SPECFEM2D_MODEL_TRUE = os.path.join(SPECFEM2D_WORKDIR, "OUTPUT_FILES_TRUE")

In [ ]:
!seisflows -h

## Populate Par_file

In [ ]:
# The command 'setup' creates the 'parameters.yaml' file that controls all of SeisFlows
# the '-f' flag removes any exist 'parameters.yaml' file that might be in the directory
os.chdir(WORKDIR)
!seisflows setup -f
!ls

In [ ]:
# Let's have a look at this file, which has not yet been populated
!cat parameters.yaml

In [ ]:
# We can use the `seisflows print modules` command to list out the available options
!seisflows print modules

In [ ]:
# For this example, we can use most of the default modules, however we need to
# change the SOLVER module to let SeisFlows know we're using SPECFEM2D (as opposed to 3D)
!seisflows par workflow inversion
!seisflows par preprocess pyaflowa
!seisflows par optimize LBFGS
!cat parameters.yaml

In [ ]:
# The seisflows configure command populates the parameter file based on the chosen modules. 
# SeisFlows will attempt to fill in all parameters with reasonable default values. 
!seisflows configure
!head --lines=50 parameters.yaml

In [ ]:
# EDIT THE SEISFLOWS PARAMETER FILE
!seisflows par ntask 3  # set the number of sources/events to use
!seisflows par nproc 1  # set the number of sources/events to use
!seisflows par materials elastic  # update Vp and Vs during inversion
!seisflows par end 5    # final iteration -- we will only run 1
!seisflows par data_case synthetic  # synthetic-synthetic means we need both INIT and TRUE models
!seisflows par components Y  # this default example creates Y-component seismograms
!seisflows par step_count_max 10  # limit the number of steps in the line search
!seisflows par smooth_h 5000  # smoothing distance 
!seisflows par smooth_v 5000  # smoothing distance
!seisflows par min_period 10  # tmin
!seisflows par max_period 200 # tmax
!seisflows par filter_corners 4 # limit the number of steps in the line search

# Use Python syntax here to access path constants
os.system(f"seisflows par path_specfem_bin {SPECFEM2D_BIN}")  # set path to SPECFEM2D binaries
os.system(f"seisflows par path_specfem_data {SPECFEM2D_DATA}")  # set path to SEPCFEM2D DATA/
os.system(f"seisflows par path_model_init {SPECFEM2D_MODEL_INIT}")  # set path to INIT model
os.system(f"seisflows par path_model_true {SPECFEM2D_MODEL_TRUE}")  # set path to TRUE model

In [ ]:
# edit the SPECFEM2D Par_file parameter MODEL such that xmeshfem2d reads our pre-built velocity models 
# (*.bin files) rather than the meshing parameters defined in the Par_file.
os.chdir(SPECFEM2D_DATA)
!seisflows sempar model gll

## Run seisflows 

### Forward simulation

In [ ]:
os.chdir(WORKDIR)
!seisflows print tasks

In the Inversion workflow, the tasks listed are described as follows:

1. Evaluate_initial_misfit:  
a. Prepare data for inversion by either copying data from disk or generating ‘synthetic data’ with MODEL_TRUE  
b. Call numerical solver to run forward simulations using MODEL_INIT, generating synthetics  
c. Evaluate the objective function by performing waveform comparisons  
d. Prepare run_adjoint_simulations step by generating adjoint sources and auxiliary files  
2. Run_adjoint_simulations: Call numerical solver to run adjoint simulation, generating kernels
3. Postprocess_event_kernels: Combine all event kernels into a misfit kernel.
4. Evaluate_gradient_from_kernels: Smooth and mask the misfit kernel to create the gradient
5. Initialize_line_search: Call on the optimization library to scale the gradient by a step length to compute the search direction. Prepare file structure for line search.
6. Perform_line_search: Perform a line search by algorithmically scaling the gradient and evaluating the misfit function (forward simulations and misfit quantification) until misfit is acceptably reduced.
7. Finalize_iteration: Run any finalization steps such as saving traces, kernels, gradients and models to disk, setting up SeisFlows for any subsequent iterations. Clean the scratch/ directory in preparation for subsequent iterations

In [ ]:
!seisflows par stop_after evaluate_initial_misfit

In [ ]:
# Now let’s run SeisFlows. There are two ways to do this: submit and restart
!seisflows submit

seisflows submit is used to run new workflows and resume stopped or failed workflows.
The restart command is simply a convenience function that runs clean (to remove an active working state) and submit (to submit a fresh workflow).
Since this is our first run, we’ll use seisflows submit.

In [ ]:
# The adjoint source is created in the same format as the synthetics (two-column ASCII)
!head scratch/solver/001/traces/adj/AA.S0001.BXY.adj

### Adjoint simulation
Now that we have all the required files for running an adjoint simulation (*.adj waveforms and STATIONS_ADJOINT file), we can continue with the SeisFlows3 Inversion workflow. No need to edit the Par_file or anything like that, SeisFlows3 will take care of that under the hood. We simply need to tell the workflow (via the parameters.yaml file) to resume_from the correct function. We can have a look at these functions again:

In [ ]:
!seisflows print tasks

In [ ]:
# We'll stop just before the line search so that we can take a look at the files
# generated during the middle tasks
!seisflows par stop_after evaluate_gradient_from_kernels

In [ ]:
# We can use the `seisflows submit` command to continue an active workflow
# The state file created during the first run will tell the workflow to resume from the stopped point in the workflow
!seisflows submit

In [ ]:
# Gradient evaluation files are stored here, the kernels are stored separately from the gradient incase
# the user wants to manually manipulate them
!ls scratch/eval_grad

In [ ]:
# SeisFlows3 stores all kernels and gradient information as SPECFEM binary (.bin) files
!ls scratch/eval_grad/gradient

In [ ]:
# Kernels are stored on a per-event basis, and summed together (sum/). If smoothing was performed,
# we would see both smoothed and unsmoothed versions of the misfit kernel
!ls scratch/eval_grad/kernels

In [ ]:
# We can see that some new values have been stored in prepartion for the line search,
# including g_new (current gradient) and p_new (current search direction). These are also
# stored as vector NumPy arrays (.npy files)
!ls scratch/optimize

In [ ]:
g_new = np.load("scratch/optimize/g_new.npz")
print(g_new["vs_kernel"])

In [ ]:
# We don't want to run the finalize_iteration argument so that we can explore the dir
!seisflows par stop_after perform_line_search 

In [ ]:
!seisflows submit

From the log statements above, we can see that the SeisFlows line search required 4 trial steps, where it modified values of Vs (shear-wave velocity) until satisfactory reduction in the objective function was met. This was the final step in the iteration, and so the finalization of the line search made preparations for a subsequent iteration.

In [ ]:
# We can see that we have 'new' and 'old' values for each of the optimization values,
# representing the previous model (M00) and the current model (M01).
!ls scratch/optimize

In [ ]:
# The stats/ directory contains text files describing the optimization/line search
!cat scratch/optimize/output_optim.txt